# DuckDB Tutorial: From Hello World to Advanced Features

This notebook contains practical examples demonstrating DuckDB's capabilities, from basic operations to advanced features like cloud data access and integration with Python data libraries.

In [ ]:
!pip install duckdb

## 1. Hello World DuckDB

Let's start with the basics - connecting to DuckDB and running a simple query.

In [ ]:
import duckdb

# Create an in-memory DuckDB connection
con = duckdb.connect()

# Run a simple query
con.sql("SELECT 'mandarin' AS breed, 6 AS age;")

## 2. DuckDB Database Files

DuckDB can work both in-memory and with persistent database files. Let's explore how tables persist across connections.

In [ ]:
# Check what tables exist in our in-memory connection
con.sql("SHOW TABLES;")

In [ ]:
# Create a table in memory
con.sql("CREATE TABLE IF NOT EXISTS ducks AS SELECT 'mandarin' AS breed, 6 AS age;")

# Now we should see the table
con.sql("SHOW TABLES;")

In [ ]:
# Let's restart fresh and create a persistent database file
import duckdb

# Connect to a file-based database (will create the file if it doesn't exist)
con = duckdb.connect('duckdb_tutorial.db')

In [ ]:
# Create a table in the persistent database
con.sql("CREATE TABLE IF NOT EXISTS ducks AS SELECT 'mandarin' AS breed, 6 AS age;")

# Show tables - this will persist even after restarting the kernel
con.sql("SHOW TABLES;")

## 3. Working with External Data Sources

DuckDB's real power comes from its ability to work with various file formats and cloud storage directly, without needing to import data first.

In [ ]:
# We can attach multiple databases and work with them simultaneously
# Note: This will create netflix.db if it doesn't exist
try:
    con.sql("ATTACH 'netflix.db' AS netflix;")
    print("Successfully attached netflix database")
except Exception as e:
    print(f"Database might already be attached: {e}")
    
# Switch to using the netflix database
con.sql("USE netflix;")

In [ ]:
# DuckDB can directly query data from cloud storage (S3) without downloading
# This is one of DuckDB's most powerful features - "zero-copy" data access
con.sql("FROM 's3://us-prd-motherduck-open-datasets/netflix/netflix_daily_top_10.parquet'")

In [ ]:
# Let's create a local table from the cloud data for easier querying
con.sql("CREATE TABLE netflix_daily_top_10 AS SELECT * FROM 's3://us-prd-motherduck-open-datasets/netflix/netflix_daily_top_10.parquet';")

In [ ]:
# DuckDB uses extensions to add functionality
# Let's see what extensions are available and loaded
con.sql("FROM duckdb_extensions()")

## 4. Data Analysis Examples

Now let's perform some analysis on the Netflix data to see DuckDB's SQL capabilities in action.


In [ ]:
# Find the TV shows that stayed in the top 10 the longest
con.sql("""
SELECT Title, max("Days In Top 10") as max_days_in_top_10
FROM netflix_daily_top_10
WHERE Type = 'TV Show'
GROUP BY Title
ORDER BY max_days_in_top_10 DESC
LIMIT 5;
""")

In [ ]:
# Now let's do the same for movies
con.sql("""
SELECT Title, max("Days In Top 10") as max_days_in_top_10
FROM netflix_daily_top_10
WHERE Type = 'Movie'
GROUP BY Title
ORDER BY max_days_in_top_10 DESC
LIMIT 5;
""")

In [ ]:
# Export query results to CSV - DuckDB makes this very easy
con.sql("""
COPY (
    SELECT Title, max("Days In Top 10") as max_days_in_top_10
    FROM netflix_daily_top_10
    WHERE Type = 'TV Show'
    GROUP BY Title
    ORDER BY max_days_in_top_10 DESC
    LIMIT 5
) TO 'top_tv_shows.csv' (HEADER, DELIMITER ',');
""")

## 5. Working with JSON APIs

DuckDB can directly query JSON data from web APIs - no need for separate HTTP libraries!


In [ ]:
# Query GitHub API to get programming languages used in DuckDB repository
# UNPIVOT transforms columns into rows - very useful for JSON data
con.sql("""
UNPIVOT read_json_auto('https://api.github.com/repos/duckdb/duckdb/languages')
ON COLUMNS(*)
INTO 
    NAME language
    VALUE bytes
ORDER BY bytes DESC;
""")


## 6. Database Connections and Secrets

DuckDB can connect to other databases like PostgreSQL, MySQL, and SQLite. Here's how to manage connections securely.

In [ ]:
# Example: Creating a secret for PostgreSQL connection
# Note: In production, never hardcode credentials like this!
# Use environment variables or secure credential management instead

con.sql("""
CREATE OR REPLACE SECRET postgres_secret (
    TYPE postgres,
    HOST 'your-postgres-host.com',
    PORT 5432,
    DATABASE your_database,
    USER 'your_username',
    PASSWORD 'your_password'
);
""")


In [ ]:
# Attach the PostgreSQL database using the secret
# This would allow you to query PostgreSQL tables directly from DuckDB
# con.sql("ATTACH '' AS postgres_db (TYPE postgres, SECRET postgres_secret);")

print("Secret created successfully (commented out for demo purposes)")


## 7. Integration with Python Data Libraries

One of DuckDB's greatest strengths is seamless integration with pandas, Polars, and other Python data libraries.


In [ ]:
!pip install polars pyarrow

In [ ]:
import duckdb
import polars as pl

# Create a Polars DataFrame
df = pl.DataFrame({
    "A": [1, 2, 3, 4, 5],
    "fruits": ["banana", "banana", "apple", "apple", "banana"],
    "B": [5, 4, 3, 2, 1],
    "cars": ["beetle", "audi", "beetle", "beetle", "beetle"],
})

# DuckDB can directly query Polars DataFrames!
# The DataFrame is automatically detected and can be used in SQL
result = duckdb.sql("""
    SELECT fruits, 
           COUNT(*) as count, 
           AVG(A) as avg_A
    FROM df 
    GROUP BY fruits
    ORDER BY count DESC
""")

result.show()


## Scaling with MotherDuck

In [ ]:
con.sql("""ATTACH 'md:'""")


In [ ]:
con.sql("""
ATTACH IF NOT EXISTS 'md:_share/hacker_news/de11a0e3-9d68-48d2-ac44-40e07a1d496b' AS hacker_news;""")
con.sql("""USE hacker_news;""")

In [ ]:
con.sql("""SHOW TABLES;""")
con.sql("""SELECT count(*) FROM hacker_news.hacker_news;""")

In [ ]:
con.sql("""
SELECT
    regexp_extract(url, 'http[s]?://([^/]+)/', 1) AS domain,
    count(*) AS count
FROM hacker_news.hacker_news
WHERE url IS NOT NULL AND regexp_extract(url, 'http[s]?://([^/]+)/', 1) != ''
GROUP BY domain
ORDER BY count DESC
LIMIT 20;
""")



In [ ]:
con.sql("""CREATE DATABASE IF NOT EXISTS duckdb_tutorial FROM './duckdb_tutorial.db';""")